In [1]:
from pyspark.sql import SparkSession, functions as F, types as T

In [2]:
spark = SparkSession.builder.master('local[*]').getOrCreate()

24/02/13 16:59:25 WARN Utils: Your hostname, MacBook-Air-Stanislav.local resolves to a loopback address: 127.0.0.1; using 192.168.88.220 instead (on interface en0)
24/02/13 16:59:25 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/02/13 16:59:26 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
print(spark.version)

3.4.1


In [4]:
actor_df = spark.read.csv('./data/actor.csv', header=True, inferSchema=True)
address_df = spark.read.csv('./data/address.csv', header=True, inferSchema=True)
category_df = spark.read.csv('./data/category.csv', header=True, inferSchema=True)
city_df = spark.read.csv('./data/city.csv', header=True, inferSchema=True)
country_df = spark.read.csv('./data/country.csv', header=True, inferSchema=True)
customer_df = spark.read.csv('./data/customer.csv', header=True, inferSchema=True)
film_df = spark.read.csv('./data/film.csv', header=True, inferSchema=True)
film_actor_df = spark.read.csv('./data/film_actor.csv', header=True, inferSchema=True)
film_category_df = spark.read.csv('./data/film_category.csv', header=True, inferSchema=True)
inventory_df = spark.read.csv('./data/inventory.csv', header=True, inferSchema=True)
language_df = spark.read.csv('./data/language.csv', header=True, inferSchema=True)
payment_df = spark.read.csv('./data/payment.csv', header=True, inferSchema=True)
rental_df = spark.read.csv('./data/rental.csv', header=True, inferSchema=True)
staff_df = spark.read.csv('./data/staff.csv', header=True, inferSchema=True)
store_df = spark.read.csv('./data/store.csv', header=True, inferSchema=True)

# Домашнє завдання на тему Spark SQL

Задачі з домашнього завдання на SQL потрібно розвʼязати за допомогою Spark SQL DataFrame API.

- Дампи таблиць знаходяться в папці `data`. Датафрейми таблиць вже створені в клітинці вище.
- Можете створювати стільки нових клітинок, скільки вам необхідно.
- Розвʼязок кожної задачі має бути відображений в самому файлі (використати метод `.show()`)
- код має бути оформлений у відповідності із одним із стилем, показаним лектором на занятті 13.

**Увага!**
Використовувати мову запитів SQL безпосередньо забороняється, потрібно використовувати виключно DataFrame API!


1.
Вивести кількість фільмів в кожній категорії.
Результат відсортувати за спаданням.

In [5]:
film_category_df.join(category_df, on='category_id', how='left').groupby('name').count().\
select(
    F.col('name').alias("category_name"),
    F.col('count').alias('number_films')
).orderBy(F.col('number_films').desc()).show()

+-------------+------------+
|category_name|number_films|
+-------------+------------+
|       Sports|          74|
|      Foreign|          73|
|       Family|          69|
|  Documentary|          68|
|    Animation|          66|
|       Action|          64|
|          New|          63|
|        Drama|          62|
|        Games|          61|
|       Sci-Fi|          61|
|     Children|          60|
|       Comedy|          58|
|       Travel|          57|
|     Classics|          57|
|       Horror|          56|
|        Music|          51|
+-------------+------------+



2.
Вивести 10 акторів, чиї фільми брали на прокат найбільше.
Результат відсортувати за спаданням.

In [19]:
rental_df.join(inventory_df, on='inventory_id', how='left').\
          join(film_actor_df, on='film_id', how='left').\
          join(actor_df, on='actor_id', how='left').\
          withColumn('actor', F.concat_ws(' ', F.col('first_name'), F.col('last_name'))).\
          groupby('actor').count().\
          select(
              F.col('actor'),
              F.col('count').alias('films_rented')
          ).orderBy(F.col('films_rented').desc()).show(10)

+------------------+------------+
|             actor|films_rented|
+------------------+------------+
|       SUSAN DAVIS|         825|
|    GINA DEGENERES|         753|
|    MATTHEW CARREY|         678|
|       MARY KEITEL|         674|
|ANGELA WITHERSPOON|         654|
|       WALTER TORN|         640|
|       HENRY BERRY|         612|
|       JAYNE NOLTE|         611|
|        VAL BOLGER|         605|
|     SANDRA KILMER|         604|
+------------------+------------+
only showing top 10 rows



3.
Вивести категорія фільмів, на яку було витрачено найбільше грошей
в прокаті

In [27]:
payment_df.join(rental_df, on='rental_id', how='left').\
           join(inventory_df, on='inventory_id', how='left').\
           join(film_category_df, on='film_id', how='left').\
           join(category_df, on='category_id', how='left').\
           groupby('name').sum('amount').\
           select(
               F.col('name'),
               F.col('sum(amount)').alias('amount')
           ).orderBy(F.col('amount').desc()).show(1)

+------+-----------------+
|  name|           amount|
+------+-----------------+
|Sports|5314.209999999847|
+------+-----------------+
only showing top 1 row



4.
Вивести назви фільмів, яких не має в inventory.

In [47]:
inventory_df_film_ids = inventory_df.select(F.col('film_id')).distinct().rdd.flatMap(lambda x: x).collect()

In [52]:
film_df.select(
    F.col('title')
).where(~F.col('film_id').isin(inventory_df_film_ids)).show()

+--------------------+
|               title|
+--------------------+
|      ALICE FANTASIA|
|         APOLLO TEEN|
|      ARGONAUTS TOWN|
|       ARK RIDGEMONT|
|ARSENIC INDEPENDENCE|
|   BOONDOCK BALLROOM|
|       BUTCH PANTHER|
|       CATCH AMISTAD|
| CHINATOWN GLADIATOR|
|      CHOCOLATE DUCK|
|COMMANDMENTS EXPRESS|
|    CROSSING DIVORCE|
|     CROWDS TELEMARK|
|    CRYSTAL BREAKING|
|          DAZED PUNK|
|DELIVERANCE MULHO...|
|   FIREHOUSE VIETNAM|
|       FLOATS GARDEN|
|FRANKENSTEIN STRA...|
|  GLADIATOR WESTWARD|
+--------------------+
only showing top 20 rows



5.
Вивести топ 3 актори, які найбільше зʼявлялись в категорії фільмів “Children”

*Збираємо датасет з імʼям актора і кількістю фільмів в категорії Children*

In [65]:
temp_df = film_category_df.join(category_df, on='category_id', how='left').\
                             join(film_df, on='film_id', how='left').\
                             join(film_actor_df, on='film_id', how='left').\
                             join(actor_df, on='actor_id', how='left').\
                             withColumn('actor_name', F.concat_ws(' ', F.col('first_name'), F.col('last_name'))).\
                             where(F.col('name') == 'Children').\
                             groupby('actor_name').count().\
                             select(
                                 F.col('actor_name'),
                                 F.col('count').alias('number_films')
                             ).orderBy(F.col('number_films').desc())

*Нумеруємо місце кожного актора*

In [82]:
from pyspark.sql import Window

w = Window.orderBy(F.col('number_films').desc())
temp_df_with_place = temp_df.select(
                        'actor_name',
                        'number_films',
                        F.row_number().over(w).alias("place")
                    )

*Так як є декілька акторів у яких кількіість фільмів дорівнює 5, то вони всі мають потрапити в топ 3. Для того щоб вивести всіх акторів - збережемо значення number_films для 3-го місця в змінну* ***third_place_value***

In [80]:
third_place_value = temp_df_with_place.select(F.col('number_films')).where(F.col('place') == 3).head()[0]

24/02/13 18:08:32 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/02/13 18:08:32 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/02/13 18:08:32 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/02/13 18:08:32 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/02/13 18:08:32 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/02/13 18:08:32 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/02/13 1

*Тепер виведемо всіх акторів у яких кількість фільмів більша або дорівнює кількості фільмів у 3го місця*

In [83]:
temp_df_with_place.select(
    'actor_name',
    'number_films'
).where(F.col('number_films') >= third_place_value).show()

+-------------+------------+
|   actor_name|number_films|
+-------------+------------+
| HELEN VOIGHT|           7|
|  SUSAN DAVIS|           6|
|   MARY TANDY|           5|
|   RALPH CRUZ|           5|
|  WHOOPI HURT|           5|
|KEVIN GARLAND|           5|
+-------------+------------+



Stop Spark session:

In [84]:
spark.stop()